In [2]:
import csv
import pandas as pd
import numpy as np
import random
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [6]:
dfFruit = pd.read_csv("buah.csv")
dfSeasoning = pd.read_csv("bumbu.csv")
dfPoultry = pd.read_csv("daging dan unggas.csv")
dfSweetnessBev = pd.read_csv("produk gula.csv")
dfSeafood = pd.read_csv("ikan, kerang, udang.csv")
dfNuts = pd.read_csv("kacang, biji, bean.csv")
dfFatOil = pd.read_csv("lemak dan minyak.csv")
dfBeverages = pd.read_csv("minuman.csv")
dfVegetables = pd.read_csv("sayuran.csv")
dfserealia = pd.read_csv("serealia dan hasil olahannya.csv")
dfMilkDairy = pd.read_csv("susu.csv")
dfEggs = pd.read_csv("telur.csv")
dfScratchTubers = pd.read_csv("umbi berpati.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'ikan, kerang, udang.csv'

In [4]:
# dfFruit.head(5)
dfFruit.insert(3,"Food Group","fruit",True)
dfFruit.insert(4,"Type","single",True)
single = True
for i in dfFruit.index:
    if(dfFruit['KODE'][i]=="OLAHAN/PRODUK/KOMPOSIT") :
        single = False
    if(single == False) :
        dfFruit.at[i, 'Type'] = "processed food"
        
dfFruit = dfFruit.dropna(subset=['KODE', 'NAMA BAHAN', 'SUMBER', 'ENERGI (Kal)', 'PROTEIN (g)', 'LEMAK (g)', 'KH (g)'])
dfFruit = dfFruit.reset_index(drop=True)
# dfFruit

In [4]:
import glob
import os

finalDf = pd.DataFrame()
path = "*.csv"
for fname in glob.glob(path):
    # print(os.path.basename(fname).split('.')[0])
    df=pd.read_csv(fname)
    df.insert(3,"Food Group",os.path.basename(fname).split('.')[0],True)
    df.insert(4,"Type","single",True)
    single = True
    for i in df.index:
        if(df['KODE'][i]=="OLAHAN/PRODUK/KOMPOSIT") :
            single = False
        if(single == False) :
            df.at[i, 'Type'] = "processed food"
            
    df = df.dropna(subset=['KODE', 'NAMA BAHAN', 'SUMBER', 'ENERGI (Kal)', 'PROTEIN (g)', 'LEMAK (g)', 'KH (g)'])
    df = df.reset_index(drop=True)
    finalDf = pd.concat([finalDf,df], ignore_index=True)

finalDf.to_csv('finalData.csv')

In [5]:
def eliminate_data(input_file, output_file, column_name, target_words):
    with open(input_file, 'r') as file_in, open(output_file, 'w', newline='') as file_out:
        reader = csv.DictReader(file_in)
        writer = csv.DictWriter(file_out, fieldnames=reader.fieldnames)
        writer.writeheader()

        for row in reader:
            # target_words = list(map(str.lower,target_words))
            if not any(word in row[column_name].lower() for word in list(map(str.lower,target_words))):
                writer.writerow(row)

input_file = 'finalData.csv'
output_file = 'finalData_eliminated.csv'
column_name = 'NAMA BAHAN'
target_words = ['tepung','mentah','darah','beras']
eliminate_data(input_file, output_file, column_name, target_words)
dfFinalEliminated = pd.read_csv("finalData_eliminated.csv")

In [6]:
dfFinalEliminated = pd.read_csv("finalData_eliminated.csv")
dfFinalEliminated.iloc[[0]]

,Unnamed: 0,KODE,NAMA BAHAN,SUMBER,Food Group,Type,AIR (g),ENERGI (Kal),PROTEIN (g),LEMAK (g),...,TEMBAGA(mg),SENG (mg),RETINOL (mcg),B-KAR (mcg),KAR -TOTAL (mcg),THIAMIN (mg),RIBOFLAVIN (mg),NIASIN (mg),VIT_C (mg),Unnamed: 24
0,0,CR001,"Kacang arab, kering",DABM-1964,biji bijian,single,11.6,330.0,23.8,1.4,...,0.85,3.4,NaN,40,140.0,0.77,0.21,1.5,2.0,100.0


In [7]:
# createPopulation
# represented two meals

def createPopulation(dfMeal, n_population):
    # pop = np.random.randint(n_meal, size=(n_genes,n_population))
    pop=[]
    for i in range(n_population) :
        # pop.append([1,1,1,0,0,0])
        sourceCarbs = dfFinalEliminated[dfFinalEliminated['Food Group']=='serealia'].sample(n=2).index
        sourceProtAnim = dfFinalEliminated[dfFinalEliminated['Food Group']=='daging dan unggas'].sample(n=2).index
        sourceVeggies = dfFinalEliminated[dfFinalEliminated['Food Group']=='sayuran'].sample(n=2).index
        pop.append([sourceCarbs[0],sourceProtAnim[0],sourceVeggies[0],sourceCarbs[1],sourceProtAnim[1],sourceVeggies[1]])
    # print(np.random.randint(8))
    pop = pd.DataFrame(pop)
    pop.columns = ['breakfast_carbSource','breakfast_protAnimSource','breakfast_veggSource','lunch_carbSource','lunch_protAnimSource','lunch_veggSource']
    # df = pd.DataFrame(data = array, columns = column_values)
    
    return pop
dfPop = createPopulation(dfFinalEliminated,5)
dfPop


,breakfast_carbSource,breakfast_protAnimSource,breakfast_veggSource,lunch_carbSource,lunch_protAnimSource,lunch_veggSource
0,713,328,616,724,385,632
1,713,394,606,735,368,549
2,772,355,581,732,354,556
3,688,401,585,784,370,561
4,721,363,592,782,360,603


In [8]:
def viewResult(df,dfFinalEliminated):
    column_name= df.columns
    for i in column_name:
        indexOfSource = df._get_value(0,i)
        print (i , " : ",dfFinalEliminated._get_value(indexOfSource, 'NAMA BAHAN') )
    # return dfFinalEliminated.iloc[[0]]['NAMA BAHAN'].Name
    
viewResult(dfPop.iloc[[0]],dfFinalEliminated)

breakfast_carbSource  :  Cangkuning
breakfast_protAnimSource  :  Punai, daging, goreng
breakfast_veggSource  :  Daun singkong
ampenan, segar
lunch_carbSource  :  Kakicak
lunch_protAnimSource  :  Sop daging sapi,
masakan
lunch_veggSource  :  Jamur merang, segar


In [9]:
def randomSelectionParent(n_population):
    position = np.random.permutation(n_population)
    # print(position)
    
    return position[0], position[1]

randomSelectionParent(5)

(0, 2)

In [10]:
def randomSelectionPosition():
  a= ['breakfast_carbSource','breakfast_protAnimSource','breakfast_veggSource','lunch_carbSource','lunch_protAnimSource','lunch_veggSource']
  n_pointSize = random.randint(1,3)
  return np.random.choice(a, size=n_pointSize, replace=False)
randomSelectionPosition()

array(['lunch_veggSource', 'lunch_carbSource'], dtype='<U24')

In [12]:
def crossover(pop,n_population):
    # multipoint crossover
    popc = pop.copy()
    for i in range(n_population):
          # inisiasi parent a dan b
        a,b = randomSelectionParent(n_population)
        swap = randomSelectionPosition()
        for i in swap :
        # crossover
             popc.iloc[a][i], popc.iloc[b][i] = pop.iloc[b][i], pop.iloc[a][i]
        # popc.iloc[i] = x
    return popc

popc=crossover(dfPop,5)

In [13]:
def mutation(popc,n_population):
    popm = popc.copy()
    mutationIn = ['breakfast_carbSource','breakfast_protAnimSource','breakfast_veggSource','lunch_carbSource','lunch_protAnimSource','lunch_veggSource']
    source=0
    for i in range(n_population):
        mutation = np.random.choice(mutationIn)
        mutation = str(mutation)
        if (mutation == 'breakfast_carbSource' or mutation == 'lunch_carbSource'):
            source = dfFinalEliminated[dfFinalEliminated['Food Group']=='serealia'].sample(n=1).index
        elif(mutation == 'breakfast_protAnimSource' or mutation == 'lunch_protAnimSource') :
            source = dfFinalEliminated[dfFinalEliminated['Food Group']=='daging dan unggas'].sample(n=1).index
        else :
            source = dfFinalEliminated[dfFinalEliminated['Food Group']=='sayuran'].sample(n=1).index
        popm.iloc[i][mutation]=source[0]
    return popm
popm = mutation(popc,5)
popm

,breakfast_carbSource,breakfast_protAnimSource,breakfast_veggSource,lunch_carbSource,lunch_protAnimSource,lunch_veggSource
0,713,328,616,726,385,632
1,713,366,581,735,354,556
2,772,355,606,784,368,576
3,758,401,585,732,360,556
4,688,394,592,782,370,571


In [160]:
# dfFinalEliminated[]
dfProcessedFood = dfFinalEliminated[dfFinalEliminated['Type'] == 'processed food']
dfProcessedFood.shape[0]

310

In [185]:
a = popm.iloc[[0]]
# a.iloc[0]['breakfast_carbSource']
a

,breakfast_carbSource,breakfast_protAnimSource,breakfast_veggSource,lunch_carbSource,lunch_protAnimSource,lunch_veggSource
0,767,358,563,710,362,585


In [14]:
def decideAmount(chromosomes):
    # Carbohydrates provide 4 calories per gram, protein provides 4 calories per gram, and fat provides 9 calories per gram.
    chromosomes = chromosomes.to_numpy()
    calNeeds = 1700
    carboNeeds = 0.6*calNeeds/4
    protNeeds = 0.15*calNeeds/4
    fatNeeds = 0.15*calNeeds/9
    amountInitiate = [
                     [2,1.5,0.6,2,1.5,0.6],
                     [1.5,1.5,0.9,2,1.5,0.6],
                     [1,1,0.5,1,1,0.5],
                     [1,0.8,0.5,0.8,0.8,0.5],
                     [3,1.2,0.8,1.3,1.5,0.8],
                     
                     
                     ]
    amountOptimal=-1
    fitnessOptimal=10000000
    finalCarbs = 0
    finalProt = 0
    finalFat = 0
    finalCal = 0
    
    for amount in amountInitiate:
        index=0
        carboTotal = 0
        protTotal = 0
        fatTotal = 0
        calTotal =0
        for i in chromosomes[0]:
            carbs = float(dfFinalEliminated.iloc[i]['KH (g)'])*amount[index]
            prot = float(dfFinalEliminated.iloc[i]['PROTEIN (g)'])*amount[index]
            fat = float(dfFinalEliminated.iloc[i]['LEMAK (g)'])*amount[index]
            cal = float(dfFinalEliminated.iloc[i]['ENERGI (Kal)'])*amount[index]
            carboTotal+=carbs
            protTotal+=prot
            calTotal+=cal
            fatTotal+=float(fat)
            index+=1
        currentFitness = abs(carboTotal-carboNeeds)+abs(calTotal-calNeeds)+abs(protNeeds-protTotal)+abs(fatNeeds-fatTotal)
        # print(currentFitness)
        if currentFitness<fitnessOptimal :
            # print(currentFitness," ", fitnessOptimal)
            amountOptimal = amount
            fitnessOptimal = currentFitness
            finalCarbs = carboTotal
            finalProt = protTotal
            finalFat = fatTotal
            finalCal = calTotal
            
        
        
    print ("kebutuhan kalori - real kalori : " ,calNeeds , " - ", finalCal)
    print ("kebutuhan karbo - real karbo : " ,carboNeeds , " - ", finalCarbs)
    print ("kebutuhan prot - real prot : " ,protNeeds , " - ", finalProt)
    print ("kebutuhan lemak - real lemak : " ,fatNeeds , " - ", fatTotal)
    print("jumlah kebutuhan : ", amountOptimal)
decideAmount(popm.iloc[[0]])

kebutuhan kalori - real kalori :  1700  -  1223.7
kebutuhan karbo - real karbo :  255.0  -  152.45
kebutuhan prot - real prot :  63.75  -  62.019999999999996
kebutuhan lemak - real lemak :  28.333333333333332  -  42.010000000000005
jumlah kebutuhan :  [3, 1.2, 0.8, 1.3, 1.5, 0.8]


In [26]:
def createPopulationAmount():
    sizeOfPopulation = 5
    sizeOfChromosomes = 6
    pop = np.random.uniform(low = 0.5, high=2, size=(sizeOfPopulation,sizeOfChromosomes))
    pop = np.round_(pop, decimals=1)
    # pop = np.round(random.uniform(0.5, 2, size=(sizeOfPopulation,sizeOfChromosomes)),2)
    # print(pop)
    pop = pd.DataFrame(pop)
    pop.columns = ['nCarbs_br','nProt_br','nVeggies_br','nCarbs_lunch','nProt_lunch','nVeggies_lunch']
    
    return pop
# createPopulationAmount()

,nCarbs_br,nProt_br,nVeggies_br,nCarbs_lunch,nProt_lunch,nVeggies_lunch
0,0.9,0.7,1.5,1.7,1.3,1.8
1,1.4,1.3,0.8,1.1,0.8,1.9
2,1.6,0.5,1.7,1.5,1.9,0.6
3,1.8,1.1,1.0,1.9,1.8,2.0
4,0.7,1.0,1.0,0.9,1.7,1.9


In [29]:
def randomSelectionParentAmount():
    sizeOfPopulation=5
    position = np.random.permutation(sizeOfPopulation)
    # print(position)
    
    return position[0], position[1]
a,b = randomSelectionParentAmount()
print(a)

[4 2 1 0 3]
4


In [33]:
import random
def randomSelectionPosition():
  a=['nCarbs_br','nProt_br','nVeggies_br','nCarbs_lunch','nProt_lunch','nVeggies_lunch']
  return np.random.choice(a, size=1, replace=False)[0]
randomSelectionPosition()

'nCarbs_lunch'

In [74]:
def crossoverAmount(pop):
    popc = pop.copy()
    sizeOfPopulation = 5
    for i in range(sizeOfPopulation):
      # inisiasi parent a dan b
        a,b = randomSelectionParent(sizeOfPopulation)
        swap = randomSelectionPosition()
        # crossover
        popc.iloc[a][swap], popc.iloc[b][swap] = pop.iloc[b][swap], pop.iloc[a][swap]
        # popc.iloc[i] = x
    return popc

In [144]:
def fitnessAmount(popAmount,chromosomes):
    # print(chromosomes)
    arrayOfAmount = popAmount.to_numpy()
    calNeeds = 1700
    carboNeeds = 0.6*calNeeds/4
    protNeeds = 0.15*calNeeds/4
    fatNeeds = 0.15*calNeeds/9
    fitnessArray=[]
    for amount in arrayOfAmount:
        # print(amount)
        index=0
        carboTotal = 0
        protTotal = 0
        fatTotal = 0
        calTotal =0
        # print(chromosomes[0])
        for i in chromosomes:
            carbs = float(dfFinalEliminated.iloc[i]['KH (g)'])*amount[index]
            prot = float(dfFinalEliminated.iloc[i]['PROTEIN (g)'])*amount[index]
            fat = float(dfFinalEliminated.iloc[i]['LEMAK (g)'])*amount[index]
            cal = float(dfFinalEliminated.iloc[i]['ENERGI (Kal)'])*amount[index]
            carboTotal+=carbs
            protTotal+=prot
            calTotal+=cal
            fatTotal+=float(fat)
            index+=1
        fitness = abs(carboTotal-carboNeeds)+abs(calTotal-calNeeds)+abs(protNeeds-protTotal)+abs(fatNeeds-fatTotal)
        fitnessArray.append(fitness)
        
    popAmount['fitness'] = fitnessArray
    return popAmount
        
# pop=createPopulationAmount() 
# chromosomes = popm.iloc[[0]]
# fitnessAmount(pop,chromosomes)

In [162]:

def decideAmountThroughGeneticAlgorithm(chromosomes):
    # Carbohydrates provide 4 calories per gram, protein provides 4 calories per gram, and fat provides 9 calories per gram.
    calNeeds = 1700
    carboNeeds = 0.6*calNeeds/4
    protNeeds = 0.15*calNeeds/4
    fatNeeds = 0.15*calNeeds/9
    goal_amount={
      'nCarbs_br': [-1], 'nProt_br': [-1],'nVeggies_br': [-1],
      'nCarbs_lunch':[-1],'nProt_lunch':[-1],'nVeggies_lunch':[-1],
      'fitness': [1000000]
      }
    goal_amount=pd.DataFrame(goal_amount)
    current_amount=pd.DataFrame()
    chromosomes = chromosomes.to_numpy()
    iteration=50
    for i in range(iteration):
        pop=createPopulationAmount()
        pop=fitnessAmount(pop,chromosomes[0])
        popc=crossoverAmount(pop)
        popc=fitnessAmount(popc,chromosomes[0])
        current_amount = popc.sort_values(by = 'fitness',ascending = True).iloc[[0]]
        if goal_amount.iloc[0]["fitness"]>current_amount.iloc[0]["fitness"]:
            goal_amount.iloc[[0]]=current_amount.iloc[[0]]
            # print(current_amount.iloc[0]["fitness"])
    # return current_amount
    return goal_amount , chromosomes
            
        
        
    print ("kebutuhan kalori - real kalori : " ,calNeeds , " - ", finalCal)
    print ("kebutuhan karbo - real karbo : " ,carboNeeds , " - ", finalCarbs)
    print ("kebutuhan prot - real prot : " ,protNeeds , " - ", finalProt)
    print ("kebutuhan lemak - real lemak : " ,fatNeeds , " - ", fatTotal)
    print("jumlah kebutuhan : ", amountOptimal)
decideAmountThroughGeneticAlgorithm(popm.iloc[[0]])

,nCarbs_br,nProt_br,nVeggies_br,nCarbs_lunch,nProt_lunch,nVeggies_lunch,fitness
0,1.9,1.7,2.0,1.9,1.9,1.8,526.776667


NameError: name 'dfFinalEliminated' is not defined